### Importing The Libraries

In [1]:
import yaml
import spacy
import plac
import random
from pathlib import Path
from spacy.util import minibatch , compounding
from spacy.training import Example

### Train Data

In [2]:
data_file = open('data//entities.yaml').read()
entities = yaml.safe_load(data_file)


train_data = []


for entity in entities['train_data']:
    train_data.append(entity)

### Create the Model

In [3]:
# Define our Variables
ner_model = None
output_dir=Path(r"C:\Users\Mohamed Nabil\Documents\GP\Name Entity Recognition")
n_iter=100

# Load the Model
if ner_model is not None:
    nlp = spacy.load(ner_model)  
    print("Loaded model '%s'" % ner_model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

# Set up the pipeline
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe('ner', last=True)
else:
    ner = nlp.get_pipe('ner')

Created blank 'en' model


### Train the Recognizer

In [4]:
for _, annotations in train_data:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

disable_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

with nlp.disable_pipes(*disable_pipes):
    optimizer = nlp.begin_training()
    
    for i in range(n_iter):
        random.shuffle(train_data)
        losses = {}
        
        batches = minibatch(train_data , size = compounding(1.0,16.0,1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            
            example = []
            # Update the model with iterating each text
            for i in range(len(texts)):
                doc = nlp.make_doc(texts[i])
                example.append(Example.from_dict(doc, annotations[i]))
            
                # Update the model
                nlp.update(
                           example,
                           drop=0.6,
                           sgd = optimizer,
                           losses=losses
                           )
            print(losses)

{'ner': 7.428570806980133}
{'ner': 13.01240336894989}
{'ner': 15.803873300552368}
{'ner': 20.396941423416138}
{'ner': 26.679881393909454}
{'ner': 30.97847455739975}
{'ner': 35.352316200733185}
{'ner': 39.597697138786316}
{'ner': 45.5924431681633}
{'ner': 50.16306644678116}
{'ner': 55.03701674938202}
{'ner': 60.93638372421265}
{'ner': 63.71878460049629}
{'ner': 69.41989198327065}
{'ner': 73.83090245723724}
{'ner': 77.97538989782333}
{'ner': 81.97358670830727}
{'ner': 84.7733262181282}
{'ner': 88.17210686206818}
{'ner': 92.14035049080849}
{'ner': 95.58119098842144}
{'ner': 98.17198541015387}
{'ner': 100.49191541969776}
{'ner': 103.57961290702224}
{'ner': 108.27459626272321}
{'ner': 111.6528383679688}
{'ner': 116.06853821873665}
{'ner': 118.8938808888197}
{'ner': 121.1402330249548}
{'ner': 123.54452254250646}
{'ner': 127.70937655307353}
{'ner': 129.58980636298656}
{'ner': 131.73499519377947}
{'ner': 135.54925264511257}
{'ner': 139.09273724351078}
{'ner': 143.39627099316567}
{'ner': 148.38

### Save the Model

In [5]:
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)   

Saved model to C:\Users\Mohamed Nabil\Documents\GP\Name Entity Recognition
